Kaggle link: https://www.kaggle.com/competitions/fake-news/overview
Score: 0.90302
Public score: 0.91602

In [1]:
import numpy as np
import pandas as pd

In [84]:
df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')


In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
df.dropna(inplace=True)
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
X = df.drop('label', axis=1)
X.shape

(18285, 4)

In [8]:
y = df['label']
y.shape

(18285,)

In [9]:
import tensorflow as tf
tf.__version__

'2.8.2'

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
### Vocabulary size (num of unique words)
voc_size=5000

## onehot Representation

In [13]:
messages = X.copy()

In [21]:
messages['title'][5]

'Jackie Mason: Hollywood Would Love Trump if He Bombed North Korea over Lack of Trans Bathrooms (Exclusive Video) - Breitbart'

In [15]:
import nltk
import re
from nltk.corpus import stopwords

In [25]:
messages.reset_index(inplace=True)

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [26]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [27]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [28]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[718, 628, 437, 4169, 3951, 4810, 3366, 1094, 3036, 1298],
 [2459, 3347, 764, 262, 1020, 4297, 4782],
 [20, 1108, 2373, 985],
 [4421, 1152, 4528, 1542, 2445, 2521],
 [3563, 1020, 2225, 3863, 536, 3709, 1020, 2252, 1725, 4839],
 [889,
  1566,
  3166,
  830,
  2720,
  253,
  3423,
  436,
  4470,
  4033,
  3251,
  642,
  2754,
  3933,
  4782],
 [3223, 574, 3842, 1769, 1041, 304, 3142, 860, 2604, 4418, 1333],
 [3049, 1652, 1494, 1557, 1193, 1326, 253, 4654, 2604, 4418, 1333],
 [1899, 4865, 4050, 3504, 400, 4869, 1177, 2911, 253, 999],
 [2269, 674, 4472, 884, 2964, 269, 2694, 2631],
 [3309, 3625, 2553, 1540, 3595, 2782, 1305, 3564, 6, 4420, 2464],
 [1542, 1222, 3951, 4869, 253, 1193],
 [4236, 2605, 666, 4344, 651, 4104, 1515, 1572, 4185],
 [10, 1766, 3023, 4910, 994, 2210, 289, 2604, 4418, 1333],
 [1985, 1518, 4286, 3939, 3700, 2604, 4418, 1333],
 [3954, 1175, 3113, 1384, 4159, 2759, 2061, 4460, 1564, 890],
 [4884, 3506, 3347],
 [3037, 4724, 3334, 4636, 253, 1, 392, 4782],
 [2188, 3917, 76

In [29]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [30]:
onehot_repr[1]

[2459, 3347, 764, 262, 1020, 4297, 4782]

## Embedding Representation

In [31]:
sent_length=20
embedded_docs = pad_sequences(onehot_repr, padding="post", maxlen=sent_length)
print(embedded_docs)

[[ 718  628  437 ...    0    0    0]
 [2459 3347  764 ...    0    0    0]
 [  20 1108 2373 ...    0    0    0]
 ...
 [2406  644 3709 ...    0    0    0]
 [3884 1193  837 ...    0    0    0]
 [2530 1202 4583 ...    0    0    0]]


In [32]:
##creating model
embedding_vector_features = 40 ##feature representation
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [36]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=50,batch_size=64)

Epoch 1/50
192/192 [==============================] - 2s 11ms/step - loss: 0.0087 - accuracy: 0.9972 - val_loss: 0.5568 - val_accuracy: 0.9074
Epoch 2/50
192/192 [==============================] - 1s 7ms/step - loss: 0.0040 - accuracy: 0.9987 - val_loss: 0.6128 - val_accuracy: 0.9089
Epoch 3/50
192/192 [==============================] - 1s 7ms/step - loss: 0.0065 - accuracy: 0.9978 - val_loss: 0.5861 - val_accuracy: 0.9042
Epoch 4/50
192/192 [==============================] - 1s 7ms/step - loss: 0.0061 - accuracy: 0.9979 - val_loss: 0.5633 - val_accuracy: 0.9082
Epoch 5/50
192/192 [==============================] - 1s 7ms/step - loss: 0.0118 - accuracy: 0.9964 - val_loss: 0.5301 - val_accuracy: 0.9054
Epoch 6/50
192/192 [==============================] - 1s 7ms/step - loss: 0.0050 - accuracy: 0.9981 - val_loss: 0.6052 - val_accuracy: 0.9026
Epoch 7/50
192/192 [==============================] - 1s 7ms/step - loss: 0.0065 - accuracy: 0.9977 - val_loss: 0.5065 - val_accuracy: 0.9075
Epoch

In [45]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.4)) #reduce overfitting
model.add(LSTM(100))
model.add(Dropout(0.4))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [46]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=50,batch_size=64)

Epoch 1/50
192/192 [==============================] - 3s 9ms/step - loss: 0.3251 - accuracy: 0.8397 - val_loss: 0.2069 - val_accuracy: 0.9105
Epoch 2/50
192/192 [==============================] - 1s 7ms/step - loss: 0.1567 - accuracy: 0.9411 - val_loss: 0.2044 - val_accuracy: 0.9215
Epoch 3/50
192/192 [==============================] - 1s 7ms/step - loss: 0.1178 - accuracy: 0.9565 - val_loss: 0.2255 - val_accuracy: 0.9155
Epoch 4/50
192/192 [==============================] - 1s 7ms/step - loss: 0.0970 - accuracy: 0.9660 - val_loss: 0.2332 - val_accuracy: 0.9140
Epoch 5/50
192/192 [==============================] - 1s 7ms/step - loss: 0.0747 - accuracy: 0.9731 - val_loss: 0.3001 - val_accuracy: 0.9127
Epoch 6/50
192/192 [==============================] - 1s 7ms/step - loss: 0.0640 - accuracy: 0.9776 - val_loss: 0.2990 - val_accuracy: 0.9170
Epoch 7/50
192/192 [==============================] - 1s 7ms/step - loss: 0.0559 - accuracy: 0.9804 - val_loss: 0.2507 - val_accuracy: 0.9115
Epoch 

## Performance Metrics and Accuracy

In [47]:
y_pred=model.predict(X_test)
y_pred=np.where(y_pred > 0.7, 1,0) ##Use AUC ROC Curve for 0.6 threshold

In [48]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,y_pred)

array([[3111,  308],
       [ 230, 2386]])

In [49]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9108533554266777

In [50]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.89      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035



In [85]:
test_df.head()
test_df.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [86]:
# test_df.dropna(inplace=True)
test_df = test_df.fillna(" ")
test_df.head()


,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [87]:
test_df.shape

(5200, 4)

In [88]:
test_messages = test_df.copy()
test_messages.reset_index(inplace=True)

In [89]:

### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose

ps = PorterStemmer()

test_corpus = []
for i in range(0, len(test_messages)):
    review = re.sub('[^a-zA-Z]', ' ', test_messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    test_corpus.append(review)

In [90]:
test_onehot_repr = [one_hot(words, voc_size) for words in test_corpus]
test_onehot_repr

[[1577, 253, 1437, 4186, 141, 4034, 2223, 2617, 2604, 4418, 1333],
 [3309, 1245, 407, 4152, 4586, 1385, 2920],
 [3501, 345, 948, 1977, 4118, 1152, 2177, 4394, 3152, 2896],
 [3220, 3872, 4881, 3804, 3746, 1333, 1277, 2604, 4418, 1333],
 [298, 3062, 2647, 4383, 1269],
 [253, 2692, 2944, 4203, 764, 4507, 2341],
 [4603, 922, 4884, 1029, 994, 3309, 4636, 253, 4782],
 [1021, 388, 1706, 1253, 1934],
 [43, 722, 1902, 1698, 3467, 884, 3817],
 [],
 [2670, 4253, 1781, 4074, 253, 1656, 4286],
 [2736, 4044, 4203, 3203, 1192],
 [3350, 4289, 3537, 4903, 1913, 2798, 923],
 [3811, 2334, 3652, 1109],
 [4117, 3367, 3306, 725, 1944, 2605, 3930, 4336, 1061, 253, 3997, 2692, 1763],
 [2384, 274, 2737, 884, 4678, 3409, 4210, 2604, 4418, 1333],
 [2604,
  3399,
  3645,
  1589,
  4263,
  2807,
  4637,
  1202,
  208,
  2082,
  3743,
  2604,
  4418,
  1333],
 [1436, 1361, 3015, 3433, 3767, 1201, 3112, 4205, 4782],
 [253, 4879, 1275, 1542, 4483, 202, 3929, 4766, 764],
 [2984, 1815, 1262, 2788, 218, 1260, 1417, 2210

In [91]:
sent_length=20
test_embedded_docs = pad_sequences(test_onehot_repr, padding="post", maxlen=sent_length)
print(test_embedded_docs)

[[1577  253 1437 ...    0    0    0]
 [3309 1245  407 ...    0    0    0]
 [3501  345  948 ...    0    0    0]
 ...
 [ 287  347 4690 ...    0    0    0]
 [1542 3656  691 ...    0    0    0]
 [ 310  758  242 ...    0    0    0]]


In [92]:
testX_final=np.array(test_embedded_docs)

In [93]:
y_pred=model.predict(testX_final)

In [94]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,y_pred)

ValueError: ignored

In [64]:
from sklearn.metrics import accuracy_score
# Score: 0.90302
accuracy_score(y_test[0:4575],y_pred)

ValueError: ignored

In [65]:
sub_df = pd.read_csv("submit.csv")
sub_df.shape

(5200, 2)

In [66]:
sub_df.head()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,1
4,20804,1


In [97]:
y_pred=np.where(y_pred > 0.7, 1,0)
len(y_pred)

5200

In [70]:
5200-4575

625

In [74]:
for i in range(0, 625):
  # y_pred.append([0])
  y_pred = np.append(y_pred, [0])

In [98]:
y_pred

array([[0],
       [1],
       [0],
       ...,
       [0],
       [1],
       [0]])

In [99]:
sub_df['label'] = y_pred

In [101]:
# sub_df.save()
sub_df.to_csv('sub_df1.csv', index=False)

In [ ]:
sub_df